In [3]:
import pandas as pd
#import calculate_adj
import anndata as ad
import scanpy as sc
from operator import index
import re
from scipy.spatial.distance import pdist, squareform
import pandas as pd
import numpy as np
from scipy import sparse

In [5]:
folder_name = "/home/sunhang/Embedding/CCST/dataset/DLPFC/"
samplea_list = {151507,
 151508,
 151509,
 151510,
 151669,
 151670,
 151671,
 151672,
 151673,
 151674,
 151675,
 151676}
sample_name = str(151673)
gene_exp_data_file = folder_name + sample_name + "_DLPFC_count.csv"
gene_loc_data_file = folder_name + sample_name + "_DLPFC_col_name.csv"
#save file
npz_file = folder_name + sample_name + "_DLPFC_distacne.npz"

In [7]:
gene_csv = pd.read_csv(gene_exp_data_file,index_col= 0 )
gene_loc_data_csv = pd.read_csv(gene_loc_data_file,index_col=0)
gene_loc_data_csv = gene_loc_data_csv.fillna("None")

In [8]:
row_name = "imagerow"
col_name = "imagecol"

In [9]:
cell_loc = gene_loc_data_csv[[row_name,col_name]].values

In [10]:
distance_np = pdist(cell_loc, metric = "euclidean")
distance_np_X =squareform(distance_np)
distance_loc_csv = pd.DataFrame(index=gene_loc_data_csv.index, columns=gene_loc_data_csv.index,data = distance_np_X)

In [145]:
gene_loc_data_csv.columns

Index(['barcode', 'sample_name', 'tissue', 'row', 'col', 'imagerow',
       'imagecol', 'Cluster', 'height', 'width', 'sum_umi', 'sum_gene',
       'subject', 'position', 'replicate', 'subject_position', 'discard',
       'key', 'cell_count', 'SNN_k50_k4', 'SNN_k50_k5', 'SNN_k50_k6',
       'SNN_k50_k7', 'SNN_k50_k8', 'SNN_k50_k9', 'SNN_k50_k10', 'SNN_k50_k11',
       'SNN_k50_k12', 'SNN_k50_k13', 'SNN_k50_k14', 'SNN_k50_k15',
       'SNN_k50_k16', 'SNN_k50_k17', 'SNN_k50_k18', 'SNN_k50_k19',
       'SNN_k50_k20', 'SNN_k50_k21', 'SNN_k50_k22', 'SNN_k50_k23',
       'SNN_k50_k24', 'SNN_k50_k25', 'SNN_k50_k26', 'SNN_k50_k27',
       'SNN_k50_k28', 'GraphBased', 'Maynard', 'Martinowich', 'Layer',
       'layer_guess', 'layer_guess_reordered', 'layer_guess_reordered_short',
       'expr_chrM', 'expr_chrM_ratio', 'SpatialDE_PCA', 'SpatialDE_pool_PCA',
       'HVG_PCA', 'pseudobulk_PCA', 'markers_PCA', 'SpatialDE_UMAP',
       'SpatialDE_pool_UMAP', 'HVG_UMAP', 'pseudobulk_UMAP', 'markers_UM

In [87]:
distance_np_X.shape

(3639, 3639)

In [78]:
threshold = 10

In [79]:
0< distance_np_X

array([[False,  True,  True, ...,  True,  True,  True],
       [ True, False,  True, ...,  True,  True,  True],
       [ True,  True, False, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ..., False,  True,  True],
       [ True,  True,  True, ...,  True, False,  True],
       [ True,  True,  True, ...,  True,  True, False]])

In [80]:
num_big = np.where((0< distance_np_X)&(distance_np_X<threshold))[0].shape[0]

In [86]:
np.where((0< distance_np_X)&(distance_np_X<threshold))

(array([   0,    0,    0, ..., 3638, 3638, 3638]),
 array([ 397,  485, 1059, ..., 2307, 2506, 3402]))

In [81]:
cell_num = distance_np_X.shape[0]

In [82]:
cell_num

3639

In [83]:
str(num_big/(cell_num*2))

'2.9024457268480353'

In [84]:
num_big

21124

In [89]:
adj_matrix = np.zeros(distance_np_X.shape)

In [94]:
non_zero_point = np.where((0< distance_np_X)&(distance_np_X<threshold))

In [106]:
adj_matrix = np.zeros(distance_np_X.shape)
non_zero_point = np.where((0< distance_np_X)&(distance_np_X<threshold))
for i in range(num_big):
    x = non_zero_point[0][i]
    y = non_zero_point[1][i]
    adj_matrix[x][y] = 1 
adj_matrix  = np.float32(adj_matrix)
adj_matrix_crs = sparse.csr_matrix(adj_matrix)

In [108]:
import dgl

In [110]:
graph = dgl.from_scipy(adj_matrix_crs,eweight_name='w')

In [115]:
graph

Graph(num_nodes=3639, num_edges=21124,
      ndata_schemes={}
      edata_schemes={'w': Scheme(shape=(), dtype=torch.float32)})

In [112]:
gene_csv.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [117]:
import torch

In [120]:
graph.ndata["feat"] = torch.tensor(gene_csv.values.T)

In [121]:
adata = ad.AnnData(gene_csv.values.T, obs=distance_loc_csv, var=pd.DataFrame(index = gene_csv.index), dtype='int32')

In [123]:
sc.pp.filter_genes(adata, min_cells=5)

In [124]:
adata_X = sc.pp.normalize_total(adata, target_sum=1, exclude_highly_expressed=True, inplace=False)['X']


In [126]:
adata_X.shape

(3639, 16923)

In [128]:
min_cells = 5
pca_n_comps = 200
adata = ad.AnnData(gene_csv.values.T, obs=distance_loc_csv, var=pd.DataFrame(index = gene_csv.index), dtype='int32')
sc.pp.filter_genes(adata, min_cells=5)
adata_X = sc.pp.normalize_total(adata, target_sum=1, exclude_highly_expressed=True, inplace=False)['X']
adata_X = sc.pp.scale(adata_X)
adata_X = sc.pp.pca(adata_X, n_comps=pca_n_comps)


In [136]:
torch.tensor(adata_X)

ValueError: At least one stride in the given numpy array is negative, and tensors with negative strides are not currently supported. (You can probably work around this by making a copy of your array  with array.copy().) 

In [139]:
adata_X.copy()

array([[-9.3412457e+00, -3.6780939e+00, -9.8174489e-01, ...,
         5.7058209e-01,  1.3656854e-02,  8.8611042e-01],
       [ 1.2493225e+01,  1.2731439e+01, -3.1885879e+00, ...,
        -1.6465030e+00, -1.9118656e+00, -8.3466291e-01],
       [ 2.4679008e+01, -1.9800875e+01, -1.1990883e+01, ...,
         3.1888038e-01,  6.1051470e-01, -9.8771882e-01],
       ...,
       [-6.1442571e+00, -3.7836647e+00, -1.3205265e+00, ...,
         7.1048689e-01,  8.2400464e-02, -2.4279246e+00],
       [ 2.2798254e+01, -1.2938790e+01, -7.6685309e+00, ...,
         8.2874918e-01, -4.6153131e-01, -4.5874929e-01],
       [ 7.3554897e+00,  9.0793505e+00, -4.5935483e+00, ...,
        -2.0453322e+00,  2.3378162e+00, -1.9939527e-01]], dtype=float32)

In [146]:
from sklearn.preprocessing import LabelEncoder
 
# Creating a instance of label Encoder.
le = LabelEncoder()
 
# Using .fit_transform function to fit label
# encoder and return encoded label
label = le.fit_transform(gene_loc_data_csv['layer_guess_reordered_short'])

In [147]:
gene_loc_data_csv["lay_num"] = label

In [140]:
graph

Graph(num_nodes=3639, num_edges=21124,
      ndata_schemes={'feat': Scheme(shape=(200,), dtype=torch.float32)}
      edata_schemes={'w': Scheme(shape=(), dtype=torch.float32)})

In [148]:
graph.ndata["feat"] = torch.tensor(adata_X.copy())
num_features = graph.ndata["feat"].shape[1]
num_classes = len(set(gene_loc_data_csv.lay_num))

In [149]:
num_classes

8

In [150]:
num_features

200